## Evangelista – LLama2 7B Quantizations - GUFF/GGML, AWQ, SqueezeLLM
- GGUF. New library that replaces GGML (deprecated). Open source, more extensible and user friendly. [Ref](https://deci.ai/blog/ggml-vs-gguf-comparing-formats-amp-top-5-methods-for-running-gguf)

### Build llama.cpp
llama.cpp provides the tools to convert model to gguf and quantize it


In [ ]:
!apt update -y
!apt install build-essential cmake -y >/dev/null

# Clone llama.cpp
!if [ ! -d "llama.cpp" ]; then git clone https://github.com/ggerganov/llama.cpp.git; fi
%cd llama.cpp

# Build llama.cpp
!pip install -r requirements.txt
!make quantize
    
%reset -f
print('Done!\n')

### Logging to HuggingFace - ONLY needed if your model is gated

In [ ]:
# NOTE You ONLY need to login if your model is gated
from huggingface_hub import notebook_login
notebook_login()

### Load and Save Your Desired Model

In [ ]:
# YOUR MODEL URI BELOW
# --------------------------------------------------------------------------------
%env HF_MODEL_URI = meta-llama/Llama-2-7b-chat-hf


import torch, os
from transformers import AutoModelForCausalLM, AutoTokenizer

HF_MODEL_URI = os.environ.get('HF_MODEL_URI')
os.environ['MODEL_NAME']=os.path.basename(HF_MODEL_URI)
MODEL_NAME = os.environ.get('MODEL_NAME')

tokenizer = AutoTokenizer.from_pretrained(
    HF_MODEL_URI,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL_URI,
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

print('Saving model...')
tokenizer.save_pretrained(f"./models/{MODEL_NAME}")
model.save_pretrained(f'./models/{MODEL_NAME}')
print('Done!\n')

### Convert model to GGUF

In [ ]:
!python ./convert.py ./models/$MODEL_NAME --outfile ./models/$MODEL_NAME/$MODEL_NAME.gguf

### Quantize model to Q4_K_M
Quantizations Reference: https://github.com/ggerganov/llama.cpp/pull/1684

In [ ]:
!./quantize ./models/$MODEL_NAME/$MODEL_NAME.gguf ./models/$MODEL_NAME/$MODEL_NAME-Q4_K_M.gguf Q4_K_M 16

### Done! Now Download It

In [ ]:
!ls -la ./models/$MODEL_NAME/$MODEL_NAME-Q4_K_M.gguf